In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)


In [6]:
df = pd.read_csv("C:/Users/yuvan/OneDrive/Desktop/ML_projects/Marketing_campaign_analysis/data/marketing_data.csv")


In [7]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,342199,1985,Graduation,Together,59011.7,1,0,2012-11-17,3,0,2,45,6,0,16,3,2,5,3,4,0,0,0,0,0,0,0,Spain
1,8075450,1975,Master,Single,1730.0,1,1,2013-04-10,96,0,0,19,1,19,0,1,1,0,2,3,0,0,0,0,0,0,0,Spain
2,13664263,1978,Graduation,Married,98584.6,0,0,2014-01-11,99,920,0,339,0,115,138,3,2,1,6,3,0,0,0,0,0,0,0,Australia
3,16164787,1976,Graduation,Married,74031.5,1,0,2014-06-18,47,265,0,130,7,1,75,1,4,0,11,4,0,0,0,0,0,0,0,Spain
4,15815139,1981,Graduation,Divorced,52784.2,1,1,2014-05-20,0,30,3,55,242,0,0,1,3,2,3,6,0,0,0,0,0,0,0,Canada


In [10]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   56000 non-null  int64  
 1   Year_Birth           56000 non-null  int64  
 2   Education            56000 non-null  object 
 3   Marital_Status       56000 non-null  object 
 4   Income               56000 non-null  float64
 5   Kidhome              56000 non-null  int64  
 6   Teenhome             56000 non-null  int64  
 7   Dt_Customer          56000 non-null  object 
 8   Recency              56000 non-null  int64  
 9   MntWines             56000 non-null  int64  
 10  MntFruits            56000 non-null  int64  
 11  MntMeatProducts      56000 non-null  int64  
 12  MntFishProducts      56000 non-null  int64  
 13  MntSweetProducts     56000 non-null  int64  
 14  MntGoldProds         56000 non-null  int64  
 15  NumDealsPurchases    56000 non-null 

In [15]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])




In [16]:
df['Dt_Customer'].dtype


dtype('<M8[ns]')

In [19]:
CURRENT_YEAR = 2025
df['Age'] = CURRENT_YEAR - df['Year_Birth']
df[['Year_Birth', 'Age']].head()


,Year_Birth,Age
0,1985,40
1,1975,50
2,1978,47
3,1976,49
4,1981,44


In [20]:
df['Children'] = df['Kidhome'] + df['Teenhome']


In [21]:
spend_cols = [
    'MntWines',
    'MntFruits',
    'MntMeatProducts',
    'MntFishProducts',
    'MntSweetProducts',
    'MntGoldProds'
]

df['Total_Spend'] = df[spend_cols].sum(axis=1)


In [22]:
df[['MntWines','MntMeatProducts','MntGoldProds','Total_Spend']].head()


,MntWines,MntMeatProducts,MntGoldProds,Total_Spend
0,0,45,16,69
1,0,19,0,39
2,920,339,138,1512
3,265,130,75,478
4,30,55,0,330


In [23]:
df['Total_Purchases'] = (
    df['NumWebPurchases'] +
    df['NumCatalogPurchases'] +
    df['NumStorePurchases']
)


In [25]:
df['Customer_Tenure_Days'] = (
    pd.Timestamp('today') - df['Dt_Customer']
).dt.days


In [26]:
df = df[(df['Age'] >= 18) & (df['Age'] <= 90)]


In [27]:
income_cap = df['Income'].quantile(0.99)
df = df[df['Income'] <= income_cap]


In [28]:
df['Age_Band'] = pd.cut(
    df['Age'],
    bins=[18, 29, 39, 49, 59, 69, 90],
    labels=['18-29','30-39','40-49','50-59','60-69','70+']
)


In [29]:
df['Income_Band'] = pd.qcut(
    df['Income'],
    q=4,
    labels=['Low','Mid','High','Very High']
)


In [31]:
df.shape
df[['Age','Income','Total_Spend','Total_Purchases','Customer_Tenure_Days']].describe()


,Age,Income,Total_Spend,Total_Purchases,Customer_Tenure_Days
count,55440.000000,55440.000000,55440.000000,55440.000000,55440.000000
mean,53.308838,56642.124475,631.772060,11.011851,4535.011869
std,12.210905,33901.093198,616.557119,5.540758,220.327464
min,29.000000,1730.000000,0.000000,0.000000,4218.000000
25%,44.000000,27898.050000,102.000000,7.000000,4328.000000
50%,52.000000,58298.600000,433.000000,11.000000,4531.000000
75%,62.000000,86020.050000,1006.000000,15.000000,4738.000000
max,89.000000,113819.300000,3431.000000,37.000000,4917.000000
